# Rationing Wedge Analysis

This notebook analyzes the **rationing wedge** in economic simulation data - the gap between theoretical excess demand (`z_market`) and actual executed trades (`executed_net`).

## Economic Theory

In our spatial Walrasian simulation:
- **`z_market[g]`**: Theoretical excess demand computed from total endowments of marketplace participants
- **`executed_net[g]`**: Actual net trades executed, constrained by personal inventory at marketplace
- **Rationing wedge**: `z_market[g] - executed_net[g]` represents unmet demand/supply due to inventory constraints

## Sign Conventions
- `z_market[g] = demand - endowment` (+ = excess demand)
- `executed_net[g] = buys - sells` (+ = net buyer)
- Rationing wedge > 0: Unmet demand (buyers constrained)
- Rationing wedge < 0: Unmet supply (sellers constrained)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure for high-quality figures
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 12

## Load Simulation Data

Load Parquet files from simulation results. Expected schema:
- `round, agent_id, timestamp`
- `pos_x, pos_y, in_marketplace, distance_to_market`
- `p[g], z_market[g], executed_net[g]` for each good g
- `utility, move_cost, equivalent_variation`

In [ ]:
# Load simulation data
data_path = Path("simulation_results")  # Adjust path as needed

# Find the most recent simulation results
parquet_files = list(data_path.glob("*.parquet"))
if not parquet_files:
    print("No simulation results found. Please run a simulation first.")
    print("Example: python scripts/run_simulation.py --config config/market_access.yaml --seed 42")
else:
    # Load the most recent file
    latest_file = max(parquet_files, key=lambda p: p.stat().st_mtime)
    print(f"Loading: {latest_file}")
    
    df = pd.read_parquet(latest_file)
    print(f"Loaded {len(df)} rows from {df['round'].nunique()} rounds")
    print(f"Agents: {df['agent_id'].nunique()}, Goods: {len([c for c in df.columns if c.startswith('z_market_')])}")
    
    # Display schema
    print("\nDataFrame columns:")
    for col in sorted(df.columns):
        print(f"  {col}")

## Compute Rationing Wedges

Calculate the difference between theoretical and executed quantities for each good.

In [ ]:
# Identify goods columns
z_market_cols = [c for c in df.columns if c.startswith('z_market_')]
executed_net_cols = [c for c in df.columns if c.startswith('executed_net_')]
price_cols = [c for c in df.columns if c.startswith('p_')]

n_goods = len(z_market_cols)
print(f"Analyzing {n_goods} goods")

# Compute rationing wedges per good
for i in range(n_goods):
    z_col = f'z_market_{i}'
    exec_col = f'executed_net_{i}'
    wedge_col = f'rationing_wedge_{i}'
    
    if z_col in df.columns and exec_col in df.columns:
        df[wedge_col] = df[z_col] - df[exec_col]
    else:
        print(f"Warning: Missing columns for good {i}")

# Aggregate wedges by round
wedge_cols = [c for c in df.columns if c.startswith('rationing_wedge_')]
round_summary = df.groupby('round')[wedge_cols + z_market_cols + executed_net_cols].sum().reset_index()

print(f"\nRationing wedge summary (first 5 rounds):")
print(round_summary[['round'] + wedge_cols].head())

## Visualization: Rationing Wedges Over Time

Plot the evolution of rationing wedges across simulation rounds.

In [ ]:
# Plot rationing wedges over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Rationing Wedge Analysis: z_market vs executed_net', fontsize=16)

# Plot 1: Aggregate rationing wedge magnitude
ax1 = axes[0, 0]
total_wedge = round_summary[wedge_cols].abs().sum(axis=1)
ax1.plot(round_summary['round'], total_wedge, 'b-', linewidth=2)
ax1.set_title('Total Rationing Wedge Magnitude')
ax1.set_xlabel('Round')
ax1.set_ylabel('|z_market - executed_net|')
ax1.grid(True, alpha=0.3)

# Plot 2: Wedges by good
ax2 = axes[0, 1]
for i, col in enumerate(wedge_cols):
    ax2.plot(round_summary['round'], round_summary[col], label=f'Good {i}', linewidth=2)
ax2.set_title('Rationing Wedges by Good')
ax2.set_xlabel('Round')
ax2.set_ylabel('Rationing Wedge')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Plot 3: Theoretical vs Executed (Good 0)
ax3 = axes[1, 0]
if n_goods > 0:
    ax3.plot(round_summary['round'], round_summary['z_market_0'], 'r-', label='Theoretical (z_market)', linewidth=2)
    ax3.plot(round_summary['round'], round_summary['executed_net_0'], 'b-', label='Executed (executed_net)', linewidth=2)
    ax3.fill_between(round_summary['round'], 
                     round_summary['z_market_0'], 
                     round_summary['executed_net_0'], 
                     alpha=0.3, color='orange', label='Rationing Gap')
    ax3.set_title('Good 0: Theoretical vs Executed')
    ax3.set_xlabel('Round')
    ax3.set_ylabel('Quantity')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Plot 4: Rationing efficiency
ax4 = axes[1, 1]
if n_goods > 0:
    # Compute rationing efficiency: executed / theoretical (when theoretical != 0)
    efficiency_data = []
    for _, row in round_summary.iterrows():
        for i in range(n_goods):
            z_theoretical = row[f'z_market_{i}']
            z_executed = row[f'executed_net_{i}']
            if abs(z_theoretical) > 1e-6:  # Avoid division by zero
                efficiency = z_executed / z_theoretical
                efficiency_data.append({
                    'round': row['round'],
                    'good': i,
                    'efficiency': efficiency
                })
    
    if efficiency_data:
        eff_df = pd.DataFrame(efficiency_data)
        for good in range(n_goods):
            good_data = eff_df[eff_df['good'] == good]
            ax4.plot(good_data['round'], good_data['efficiency'], 
                    label=f'Good {good}', linewidth=2, alpha=0.7)
        
        ax4.set_title('Execution Efficiency (executed/theoretical)')
        ax4.set_xlabel('Round')
        ax4.set_ylabel('Efficiency Ratio')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        ax4.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Perfect execution')
        ax4.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

## Market Participation Analysis

Analyze how marketplace participation affects rationing patterns.

In [ ]:
# Analyze participation vs rationing
participation_stats = df.groupby('round').agg({
    'in_marketplace': ['sum', 'count'],
    'agent_id': 'nunique'
}).reset_index()

participation_stats.columns = ['round', 'agents_in_market', 'total_observations', 'total_agents']
participation_stats['participation_rate'] = participation_stats['agents_in_market'] / participation_stats['total_agents']

# Merge with rationing data
analysis_df = pd.merge(round_summary, participation_stats, on='round')
analysis_df['total_rationing'] = analysis_df[wedge_cols].abs().sum(axis=1)

# Plot participation vs rationing
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Participation rate over time
ax1.plot(analysis_df['round'], analysis_df['participation_rate'], 'g-', linewidth=2, marker='o')
ax1.set_title('Market Participation Rate')
ax1.set_xlabel('Round')
ax1.set_ylabel('Participation Rate')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.1)

# Scatter: participation vs rationing
scatter = ax2.scatter(analysis_df['participation_rate'], analysis_df['total_rationing'], 
                     c=analysis_df['round'], cmap='viridis', alpha=0.7, s=50)
ax2.set_title('Participation Rate vs Total Rationing')
ax2.set_xlabel('Market Participation Rate')
ax2.set_ylabel('Total Rationing Wedge')
ax2.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax2, label='Round')

plt.tight_layout()
plt.show()

# Correlation analysis
correlation = analysis_df['participation_rate'].corr(analysis_df['total_rationing'])
print(f"\nCorrelation between participation rate and total rationing: {correlation:.3f}")

## Welfare Impact of Rationing

Analyze how rationing wedges affect agent welfare and market efficiency.

In [ ]:
# Welfare analysis
if 'equivalent_variation' in df.columns:
    welfare_stats = df.groupby('round').agg({
        'equivalent_variation': ['mean', 'sum', 'std'],
        'utility': ['mean', 'sum'],
        'move_cost': ['mean', 'sum']
    }).reset_index()
    
    welfare_stats.columns = ['round', 'avg_ev', 'total_ev', 'ev_std', 'avg_utility', 'total_utility', 'avg_move_cost', 'total_move_cost']
    
    # Merge with rationing data
    welfare_analysis = pd.merge(analysis_df, welfare_stats, on='round')
    
    # Plot welfare metrics
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Welfare Impact of Rationing', fontsize=16)
    
    # Total welfare loss vs rationing
    ax1 = axes[0, 0]
    scatter1 = ax1.scatter(welfare_analysis['total_rationing'], welfare_analysis['total_ev'], 
                          c=welfare_analysis['round'], cmap='plasma', alpha=0.7, s=50)
    ax1.set_title('Total Rationing vs Welfare Loss (EV)')
    ax1.set_xlabel('Total Rationing Wedge')
    ax1.set_ylabel('Total Equivalent Variation')
    ax1.grid(True, alpha=0.3)
    plt.colorbar(scatter1, ax=ax1, label='Round')
    
    # Average welfare components over time
    ax2 = axes[0, 1]
    ax2.plot(welfare_analysis['round'], welfare_analysis['avg_utility'], label='Utility', linewidth=2)
    ax2.plot(welfare_analysis['round'], welfare_analysis['avg_move_cost'], label='Movement Cost', linewidth=2)
    ax2.plot(welfare_analysis['round'], welfare_analysis['avg_ev'], label='Equivalent Variation', linewidth=2)
    ax2.set_title('Average Welfare Components')
    ax2.set_xlabel('Round')
    ax2.set_ylabel('Value (numéraire units)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Welfare distribution
    ax3 = axes[1, 0]
    final_round_data = df[df['round'] == df['round'].max()]
    ax3.hist(final_round_data['equivalent_variation'], bins=20, alpha=0.7, edgecolor='black')
    ax3.set_title('Final Round EV Distribution')
    ax3.set_xlabel('Equivalent Variation')
    ax3.set_ylabel('Frequency')
    ax3.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='No welfare change')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Efficiency frontier
    ax4 = axes[1, 1]
    ax4.plot(welfare_analysis['round'], welfare_analysis['total_ev'], 'b-', linewidth=2, label='Spatial Welfare')
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Frictionless Baseline')
    ax4.set_title('Spatial Deadweight Loss Over Time')
    ax4.set_xlabel('Round')
    ax4.set_ylabel('Total EV (efficiency loss)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    final_rationing = welfare_analysis.iloc[-1]['total_rationing']
    final_welfare_loss = welfare_analysis.iloc[-1]['total_ev']
    
    print(f"\n=== FINAL SIMULATION SUMMARY ===")
    print(f"Final round total rationing wedge: {final_rationing:.4f}")
    print(f"Final round total welfare loss (EV): {final_welfare_loss:.4f} units of good 1")
    print(f"Average welfare loss per agent: {final_welfare_loss / welfare_analysis.iloc[-1]['total_agents']:.4f}")
    
    rationing_welfare_corr = welfare_analysis['total_rationing'].corr(welfare_analysis['total_ev'])
    print(f"Correlation between rationing and welfare loss: {rationing_welfare_corr:.3f}")

else:
    print("Equivalent variation data not found. Welfare analysis skipped.")

## Summary and Economic Insights

### Key Findings

1. **Rationing Wedge Magnitude**: How large are the gaps between theoretical and executed trades?
2. **Temporal Patterns**: Do rationing wedges decrease as agents converge to marketplace?
3. **Cross-Good Analysis**: Which goods experience more rationing constraints?
4. **Welfare Implications**: How does rationing translate to deadweight loss?

### Research Applications

This analysis helps validate the economic realism of the simulation:
- **Personal inventory constraints** create realistic market frictions
- **Spatial separation** generates measurable efficiency losses
- **Carry-over effects** show how unmet demand propagates across rounds
- **Welfare measurement** quantifies the cost of spatial frictions in numéraire units

### Next Steps

1. Run simulations with different movement costs (κ) to study sensitivity
2. Compare rationing patterns across different market sizes and agent densities
3. Analyze convergence properties: do rationing wedges vanish as T → ∞?
4. Test policy interventions: how do marketplace size changes affect rationing?

In [ ]:
# Save analysis results
if 'df' in locals():
    # Export summary statistics
    summary_stats = {
        'total_rounds': df['round'].nunique(),
        'total_agents': df['agent_id'].nunique(),
        'n_goods': n_goods,
        'final_rationing': analysis_df.iloc[-1]['total_rationing'] if len(analysis_df) > 0 else None,
        'max_rationing': analysis_df['total_rationing'].max() if len(analysis_df) > 0 else None,
        'avg_participation': analysis_df['participation_rate'].mean() if len(analysis_df) > 0 else None
    }
    
    print("\n=== SIMULATION SUMMARY STATISTICS ===")
    for key, value in summary_stats.items():
        print(f"{key}: {value}")
    
    # Save processed data for further analysis
    output_path = Path("analysis_results")
    output_path.mkdir(exist_ok=True)
    
    analysis_df.to_csv(output_path / "rationing_analysis.csv", index=False)
    print(f"\nAnalysis results saved to {output_path / 'rationing_analysis.csv'}")
    
    # Save summary statistics
    import json
    with open(output_path / "summary_stats.json", "w") as f:
        json.dump(summary_stats, f, indent=2)
    
    print("Analysis complete! Results saved for further research.")
else:
    print("No simulation data loaded. Please check file paths and run the data loading cells.")